<a href="https://colab.research.google.com/github/mori8/NLP-Pytorch-practice/blob/main/pytorch_lecture/Chapter_12_RNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys

In [19]:
idx2char = ['h', 'i', 'e', 'l', 'o']

x_data = [0, 1, 0, 2, 3, 3]   # hihell
one_hot_lookup = [[1, 0, 0, 0, 0],  # 0
                  [0, 1, 0, 0, 0],  # 1
                  [0, 0, 1, 0, 0],  # 2
                  [0, 0, 0, 1, 0],  # 3
                  [0, 0, 0, 0, 1]]  # 4

y_data = [1, 0, 2, 3, 3, 4]    # ihello
x_one_hot = [one_hot_lookup[x] for x in x_data]

inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

In [20]:
num_classes = 5
input_size = 5      # one-hot size
hidden_size = 5     # output from the LSTM. 5 to directly predict one-hot
batch_size = 1      # one sentence
sequence_length = 1 # Let's do one by one
num_layers = 1      # one-layer RNN

In [33]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
  
  def forward(self, hidden, x):
    # Reshape input in (batch_size, sequence_length, input_size)
    x = x.view(batch_size, sequence_length, input_size)
    
    # Propagate input through RNN
    # Input: (batch, seq_len, input_size)
    # Hidden: (num_layers * num_directions, batch, hidden_size)
    out, hidden = self.rnn(x, hidden)
    out = out.view(-1, num_classes)  # To compute Loss
    return hidden, out
  
  def init_hidden(self):
    # Initialize hidden and cell states
    # (num_layers * num_directions, batch, hidden_size)
    return Variable(torch.zeros(num_layers, batch_size, hidden_size))

In [34]:
model = Model()
print(model)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Train the model
for epoch in range(100):
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()

    sys.stdout.write("predicted string: ")
    for input, label in zip(inputs, labels):
        # print(input.size(), label.size())
        hidden, output = model(hidden, input)
        val, idx = output.max(1)
        sys.stdout.write(idx2char[idx.data[0]])
        loss += criterion(output, torch.LongTensor([label]))

    print(", epoch: %d, loss: %1.3f" % (epoch + 1, loss))

    loss.backward()
    optimizer.step()

print("Learning finished!")

Model(
  (rnn): RNN(5, 5, batch_first=True)
)
predicted string: iiiiii, epoch: 1, loss: 10.793
predicted string: llliii, epoch: 2, loss: 9.333
predicted string: llllil, epoch: 3, loss: 8.436
predicted string: llello, epoch: 4, loss: 7.637
predicted string: llello, epoch: 5, loss: 6.934
predicted string: llello, epoch: 6, loss: 6.316
predicted string: llello, epoch: 7, loss: 5.749
predicted string: ilello, epoch: 8, loss: 5.266
predicted string: ilello, epoch: 9, loss: 4.875
predicted string: ilello, epoch: 10, loss: 4.582
predicted string: ilello, epoch: 11, loss: 4.363
predicted string: ihello, epoch: 12, loss: 4.178
predicted string: ihello, epoch: 13, loss: 4.008
predicted string: ihello, epoch: 14, loss: 3.861
predicted string: ihello, epoch: 15, loss: 3.741
predicted string: ihello, epoch: 16, loss: 3.663
predicted string: ihello, epoch: 17, loss: 3.591
predicted string: ihello, epoch: 18, loss: 3.494
predicted string: ihello, epoch: 19, loss: 3.391
predicted string: ihello, epoch